In [1]:
import os 
import fastf1
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# enable cache to locally store data, so that when load data don't need to take them every time from the api 
cache_dir = os.path.expanduser('~/fastf1_cache')
os.makedirs(cache_dir, exist_ok=True)
fastf1.Cache.enable_cache(cache_dir)

# I don't know if this is needed considering what we are doing now 

In [3]:
# consider doing some work on data before actually appending it to the csv -> take from 3 practice session the fastest lap, 
# try to estimate the average time during a stint in practice 
# also add to dataset the Results data -> using SessionResults

In [4]:
def save_driver_session_data(new_df, event_name, year):
    output_folder = "./data"
    output_file = os.path.join(output_folder, f"f1_data_{event_name.replace(' ', '_')}_{year}.csv")

    os.makedirs(output_folder, exist_ok=True)

    if os.path.isfile(output_file):
        # Load existing data
        existing_df = pd.read_csv(output_file)

        # Merge on Driver, keeping all data
        combined_df = pd.merge(
            existing_df,
            new_df,
            on='Driver',
            how='outer',
            suffixes=('', '_dup')
        )

        # Remove duplicate columns caused by merge (e.g., if same column already existed)
        combined_df = combined_df.loc[:, ~combined_df.columns.str.endswith('_dup')]

    else:
        combined_df = new_df

    # Save the merged result
    combined_df.to_csv(output_file, index=False)
    print(f"Data saved to: {output_file}")

In [5]:
def get_practice_data(year, event_name, session_name="FP1"):

    # Load session data
    session = fastf1.get_session(year, event_name, session_name)
    session.load(laps=True)
    laps = session.laps.copy()

    # Get fastest lap for each driver using IsPersonalBest
    fastest_laps = (
        laps[laps["IsPersonalBest"] == True]
        .sort_values("LapTime")
        .drop_duplicates(subset=["Driver"], keep="first")
        .reset_index(drop=True)
        [["Driver", "LapTime", "Compound", "SpeedST", "SpeedFL", "SpeedI1", "SpeedI2"]]
        .rename(columns=lambda x: x if x == 'Driver' else f"{x}_{session_name}")
    )

    # Get weather data for each lap
    weather_data = laps.get_weather_data().reset_index(drop=True)
    laps_weather = pd.concat([laps.reset_index(drop=True), weather_data.drop(columns=['Time'])], axis=1)

    stint_data = []
    threshold_percentage = 1.05  # Slightly more forgiving threshold

    for driver in laps['Driver'].unique():
        driver_laps = laps_weather[laps_weather['Driver'] == driver]

        # Skip if driver has no valid laps
        if driver_laps.empty or driver_laps['LapTime'].dropna().empty:
            continue

        # Identify and exclude stint with driver's fastest lap
        fastest_lap_row = driver_laps.loc[driver_laps['LapTime'].idxmin()]
        fastest_lap_stint = fastest_lap_row['Stint']
        potential_stints = driver_laps[driver_laps['Stint'] != fastest_lap_stint]

        # Consider only stints with minimum laps (e.g., at least 5 laps)
        stint_counts = (
            potential_stints.groupby(['Stint', 'Compound'])
            .size()
            .reset_index(name='Laps')
            .query("Laps >= 5")  # Minimum realistic stint length
            .sort_values('Laps', ascending=False)
        )

        if stint_counts.empty:
            continue

        longest_stint = stint_counts.iloc[0]

        # Select laps from identified longest stint, excluding deleted laps
        stint_laps = potential_stints[
            (potential_stints['Stint'] == longest_stint['Stint']) &
            (potential_stints['Compound'] == longest_stint['Compound']) &
            (~potential_stints['Deleted'])
        ].sort_values('LapNumber')

        if stint_laps.empty:
            continue

        # Exclude slow laps (cooldown laps)
        fastest_stint_lap = stint_laps['LapTime'].min()
        cutoff_time = fastest_stint_lap * threshold_percentage
        valid_stint_laps = stint_laps[stint_laps['LapTime'] <= cutoff_time]

        # Ensure laps remain after filtering; otherwise, adjust logic
        if valid_stint_laps.empty:
            continue

        # Compute metrics clearly and reliably
        avg_stint_time = valid_stint_laps['LapTime'].mean()
        delta_time = valid_stint_laps['LapTime'].iloc[-1] - valid_stint_laps['LapTime'].iloc[0]
        invalid_laps = stint_laps.shape[0] - valid_stint_laps.shape[0]

        stint_data.append({
            'Driver': driver,
            f'LongestStintCompound_{session_name}': longest_stint['Compound'],
            f'LongestStintLaps_{session_name}': longest_stint['Laps'],
            f'AvgLapTime_LongestStint_{session_name}': avg_stint_time,
            f'Delta_FirstLastLap_{session_name}': delta_time,
            f'InvalidLaps_LongestStint_{session_name}': invalid_laps,
            f'AvgTyreLife_LongestStint_{session_name}': valid_stint_laps['TyreLife'].mean(),
            f'StartTyreLife_LongestStint_{session_name}': valid_stint_laps['TyreLife'].iloc[0],
            f'AvgAirTemp_LongestStint_{session_name}': valid_stint_laps['AirTemp'].mean(),
            f'AvgTrackTemp_LongestStint_{session_name}': valid_stint_laps['TrackTemp'].mean(),
            f'AvgWindSpeed_LongestStint_{session_name}': valid_stint_laps['WindSpeed'].mean(),
            f'AvgWindDirection_LongestStint_{session_name}': valid_stint_laps['WindDirection'].mean(),
            f'AvgHumidity_LongestStint_{session_name}': valid_stint_laps['Humidity'].mean(),
            f'AvgPressure_LongestStint_{session_name}': valid_stint_laps['Pressure'].mean()
        })

    stint_df = pd.DataFrame(stint_data)

    # Keep only the fastest personal best lap weather per driver
    fastest_weather = laps_weather[laps_weather["IsPersonalBest"] == True].sort_values('LapTime').drop_duplicates(subset=['Driver'], keep='first')[
        ['Driver', 'AirTemp', 'TrackTemp', 'WindSpeed', 'WindDirection', 'Humidity', 'Pressure']
    ].rename(columns=lambda x: x if x == 'Driver' else f"{x}_FastestLap_{session_name}")

    # Columns that should be present in stint_df
    stint_df_columns = [
        'Driver',
        f'LongestStintCompound_{session_name}',
        f'LongestStintLaps_{session_name}',
        f'AvgLapTime_LongestStint_{session_name}',
        f'Delta_FirstLastLap_{session_name}',
        f'InvalidLaps_LongestStint_{session_name}',
        f'AvgTyreLife_LongestStint_{session_name}',
        f'StartTyreLife_LongestStint_{session_name}',
        f'AvgAirTemp_LongestStint_{session_name}',
        f'AvgTrackTemp_LongestStint_{session_name}',
        f'AvgWindSpeed_LongestStint_{session_name}',
        f'AvgWindDirection_LongestStint_{session_name}',
        f'AvgHumidity_LongestStint_{session_name}',
        f'AvgPressure_LongestStint_{session_name}'
    ]

    # Check if stint_df is empty, and create an empty DataFrame if so
    if stint_df.empty:
        stint_df = pd.DataFrame(columns=stint_df_columns)

    # Similarly, ensure fastest_laps and fastest_weather have Driver columns
    if fastest_laps.empty:
        fastest_laps = pd.DataFrame(columns=['Driver'])
    if fastest_weather.empty:
        fastest_weather = pd.DataFrame(columns=['Driver'])


    # Merge fastest laps, stint data, and weather data
    driver_session_summary = fastest_laps.merge(stint_df, on='Driver', how='left')
    driver_session_summary = driver_session_summary.merge(fastest_weather, on='Driver', how='left')

    save_driver_session_data(driver_session_summary, event_name, year)

In [6]:
def get_quali_data(year, event_name):
    session = fastf1.get_session(year, event_name, 'Q')
    session.load(laps=True)
    results = session.results

    qualifying_data = []

    weather_data = session.laps.get_weather_data().reset_index(drop=True)
    laps_weather = pd.concat([session.laps.reset_index(drop=True), weather_data.drop(columns=['Time'])], axis=1)

    for _, row in results.iterrows():
        driver = row['Abbreviation']

        driver_info = {
            'Driver': driver,
            'Q1_FastestLap': row['Q1'],
            'Q2_FastestLap': row['Q2'],
            'Q3_FastestLap': row['Q3'],
            'QualifyingPosition': row['Position']
        }

        for q, q_time in zip(['Q1', 'Q2', 'Q3'], [row['Q1'], row['Q2'], row['Q3']]):
            if pd.isnull(q_time):
                driver_info[f'{q}_TopSpeedST'] = None
                driver_info[f'{q}_AirTemp'] = None
                driver_info[f'{q}_TrackTemp'] = None
                driver_info[f'{q}_WindSpeed'] = None
                driver_info[f'{q}_WindDirection'] = None
                driver_info[f'{q}_Humidity'] = None
                driver_info[f'{q}_Pressure'] = None
            else:
                lap_data = laps_weather[(laps_weather['Driver'] == driver) & (laps_weather['LapTime'] == q_time)]
                if not lap_data.empty:
                    lap = lap_data.iloc[0]
                    driver_info[f'{q}_TopSpeedST'] = lap['SpeedST']
                    driver_info[f'{q}_AirTemp'] = lap['AirTemp']
                    driver_info[f'{q}_TrackTemp'] = lap['TrackTemp']
                    driver_info[f'{q}_WindSpeed'] = lap['WindSpeed']
                    driver_info[f'{q}_WindDirection'] = lap['WindDirection']
                    driver_info[f'{q}_Humidity'] = lap['Humidity']
                    driver_info[f'{q}_Pressure'] = lap['Pressure']
                else:
                    driver_info[f'{q}_TopSpeedST'] = None
                    driver_info[f'{q}_AirTemp'] = None
                    driver_info[f'{q}_TrackTemp'] = None
                    driver_info[f'{q}_WindSpeed'] = None
                    driver_info[f'{q}_WindDirection'] = None
                    driver_info[f'{q}_Humidity'] = None
                    driver_info[f'{q}_Pressure'] = None

        qualifying_data.append(driver_info)

    qualifying_df = pd.DataFrame(qualifying_data)

    save_driver_session_data(qualifying_df, event_name, year)

    

In [7]:
def get_race_result(year, event_name):
    # Load race session
    session = fastf1.get_session(year, event_name, 'Race')
    session.load()

    # Extract relevant data
    results = session.results[["Abbreviation", "Time", "Position", "TeamName", ]].copy()
    results.rename(columns={
        "Time": "TotalRaceTime",
        "Position": "RacePosition",
        "Abbreviation": "Driver"
    }, inplace=True)

    status_series = session.track_status['Status']
    sc_transitions = (status_series.shift(1) != '4') & (status_series == '4')
    vsc_transitions = (status_series.shift(1) != '6') & (status_series == '6')

    sc_count = sc_transitions.sum()
    vsc_count = vsc_transitions.sum()

   

    results["SafetyCars"] = sc_count
    results["VirtualSafetyCars"] = vsc_count
    # Merge and save
    save_driver_session_data(results, event_name, year)

In [8]:
def load_event_data(year, event_name):
    get_practice_data(year, event_name, "FP1")
    get_practice_data(year, event_name, "FP2")
    get_practice_data(year, event_name, "FP3")
    get_quali_data(year, event_name)
    get_race_result(year, event_name) 

In [9]:
#load_event_data(2025, "Australian Grand Prix")
#load_event_data(2025, "Japanese Grand Prix")
#load_event_data(2025, "Bahrain Grand Prix")

In [10]:
# load 20024 race data for races which were not sprint weekends
#load_event_data(2024, "Bahrain Grand Prix")
#load_event_data(2024, "Saudi Arabian Grand Prix")
#load_event_data(2024, "Australian Grand Prix")
#load_event_data(2024, "Japanese Grand Prix")
#load_event_data(2024, "Emilia Romagna Grand Prix")
#load_event_data(2024, "Monaco Grand Prix")
#load_event_data(2024, "Canadian Grand Prix")
#load_event_data(2024, "Spain Grand Prix")
#load_event_data(2024, "British Grand Prix")
#load_event_data(2024, "Hungarian Grand Prix")
#load_event_data(2024, "Belgian Grand Prix")
#load_event_data(2024, "Dutch Grand Prix")
#load_event_data(2024, "Italian Grand Prix")
#load_event_data(2024, "Azerbaijan Grand Prix")
#load_event_data(2024, "Singapore Grand Prix")
#load_event_data(2024, "Mexican Grand Prix")
#load_event_data(2024, "Las Vegas Grand Prix")
#load_event_data(2024, "Abu Dhabi Grand Prix")

In [11]:
#load 2023 race data, without sprint weekends
#load_event_data(2023, "Bahrain Grand Prix")
#load_event_data(2023, "Saudi Arabian Grand Prix")
#load_event_data(2023, "Australian Grand Prix")
#load_event_data(2023, "Miami Grand Prix")
#load_event_data(2023, "Monaco Grand Prix")
#load_event_data(2023, "Spanish Grand Prix")
#load_event_data(2023, "Canadian Grand Prix")
#load_event_data(2023, "British Grand Prix")
#load_event_data(2023, "Hungarian Grand Prix")
#load_event_data(2023, "Dutch Grand Prix")
#load_event_data(2023, "Italian Grand Prix")
#load_event_data(2023, "Singapore Grand Prix")
#load_event_data(2023, "Japanese Grand Prix")
#load_event_data(2023, "Mexican Grand Prix")
#load_event_data(2023, "Las Vegas Grand Prix")
#load_event_data(2023, "Abu Dhabi Grand Prix")

In [ ]:
# 2022 data 
#load_event_data(2022, "Bahrain Grand Prix")
#load_event_data(2022, "Saudi Arabian Grand Prix")
#load_event_data(2022, "Australian Grand Prix")
#load_event_data(2022, "Miami Grand Prix")
#load_event_data(2022, "Spanish Grand Prix")
#load_event_data(2022, "Monaco Grand Prix")
#load_event_data(2022, "Azerbaijan Grand Prix")
#load_event_data(2022, "Canadian Grand Prix")
#load_event_data(2022, "British Grand Prix")
#load_event_data(2022, "France Grand Prix")
#load_event_data(2022, "Hungarian Grand Prix")
#load_event_data(2022, "Belgian Grand Prix")
#load_event_data(2022, "Dutch Grand Prix")
#load_event_data(2022, "Italian Grand Prix")
#load_event_data(2022, "Singapore Grand Prix")
#load_event_data(2022, "Japanese Grand Prix")
#load_event_data(2022, "United States Grand Prix")
#load_event_data(2022, "Mexican Grand Prix")
#load_event_data(2022, "Abu Dhabi Grand Prix")

core           INFO 	Loading data for Abu Dhabi Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['3', '5', '10', '11', '16', '20', '22', '23', '28', '31', '34', '36', '39', '44', '45', '51', '63', '77', '82', '88']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Practice 2 [v3.5.3]
req            INFO 	Usi

Data saved to: ./data/f1_data_Abu_Dhabi_Grand_Prix_2022.csv


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to cache!
core           INFO 	Processing timing data...
req            INFO 	No cached data found for car_data. Loading data...
_api           INFO 	Fetching car data...
_api           INFO 	Parsing car data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	F

Data saved to: ./data/f1_data_Abu_Dhabi_Grand_Prix_2022.csv


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

Data saved to: ./data/f1_data_Abu_Dhabi_Grand_Prix_2022.csv


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

Data saved to: ./data/f1_data_Abu_Dhabi_Grand_Prix_2022.csv


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

Data saved to: ./data/f1_data_Abu_Dhabi_Grand_Prix_2022.csv


In [ ]:
# 2021 data 
#load_event_data(2021, "Bahrain Grand Prix")
#load_event_data(2021, "Emilia Romagna Grand Prix")
#load_event_data(2021, "Portuguese Grand Prix")
#load_event_data(2021, "Spanish Grand Prix")
#load_event_data(2021, "Monaco Grand Prix")
#load_event_data(2021, "Azerbaijan Grand Prix")
#load_event_data(2021, "French Grand Prix")
load_event_data(2021, "Austrian Grand Prix")
load_event_data(2021, "Styrian Grand Prix")
load_event_data(2021, "Hungarian Grand Prix")
load_event_data(2021, "Belgian Grand Prix")
load_event_data(2021, "Dutch Grand Prix")
load_event_data(2021, "Russian Grand Prix")
load_event_data(2021, "Turkish Grand Prix")
load_event_data(2021, "United States Grand Prix")
load_event_data(2021, "Mexican Grand Prix")
load_event_data(2021, "Qatar Grand Prix")
load_event_data(2021, "Saudi Arabian Grand Prix")
load_event_data(2021, "Abu Dhabi Grand Prix")

core           INFO 	Loading data for Bahrain Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['3', '4', '5', '6', '7', '9', '10', '11', '14', '16', '18', '22', '31', '33', '44', '47', '55', '63', '77', '99']
core           INFO 	Loading data for Bahrain Grand Prix - Practice 2 [v3.5.3]
req            INFO 	Using cache

Data saved to: ./data/f1_data_Bahrain_Grand_Prix_2021.csv


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['3', '4', '5', '6', '7', '9', '10', '11', '14', '16', '18', '22', '31', '33', '44', '47', '55', '63', '77', '99']
core           INFO 	Loading data for Bahrain Grand Prix - Practice 3 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Data saved to: ./data/f1_data_Bahrain_Grand_Prix_2021.csv


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['3', '4', '5', '6', '7', '9', '10', '11', '14', '16', '18', '22', '31', '33', '44', '47', '55', '63', '77', '99']
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


...

Data saved to: ./data/f1_data_Austrian_Grand_Prix_2021.csv


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

Data saved to: ./data/f1_data_Austrian_Grand_Prix_2021.csv


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
logger      WARNING 	Failed to load timing data!
req            INFO 	No cached data found for car_data. Loading data...
_api           INFO 	Fetching car data...
logger      WARNING 	Failed to load telemetry data!
req

DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`